In [1]:
from pathlib import Path

import pandas as pd
from neptoon.quality_control import (
    QualityAssessmentFlagBuilder,
    QualityCheck,
    QATarget,
    QAMethod,
)
from neptoon.corrections import (
    CorrectionType,
    CorrectionTheory,
)
from neptoon.calibration import CalibrationConfiguration
from neptoon.data_audit import (
    DataAuditLog,
)
from neptoon.io.read import (
    ConfigurationManager
)
from neptoon.workflow import ProcessWithYaml

In [ ]:
# DataAuditLog.create()

In [2]:
config = ConfigurationManager()
station_config_path = (
    Path.cwd() / "configuration_files" / "FSC001_station.yaml"
)
processing_config_path = (
    Path.cwd() / "configuration_files" / "v1_processing_method.yaml"
)

config.load_configuration(
    file_path=station_config_path,
)
config.load_configuration(
    file_path=processing_config_path,
)

yaml_processor = ProcessWithYaml(configuration_object=config)

## OPTION 1:
data_hub = yaml_processor.create_data_hub()

In [3]:
data_hub.crns_data_frame

,epithermal_neutrons_raw,NeutronCount_Slow_Cum1h,AirTemperature,AirHumidity_Relative,air_pressure,Precipitation,air_temperature,air_relative_humidity,epithermal_neutrons_cph
DateTime_utc,,,,,,,,,
2014-01-09 11:00:00+00:00,655.0,NaN,NaN,NaN,996.2,NaN,8.9,93.9,655.0
2014-01-09 12:00:00+00:00,715.0,431.0,NaN,NaN,994.3,NaN,10.1,84.4,715.0
2014-01-09 13:00:00+00:00,741.0,473.0,NaN,NaN,993.6,NaN,10.6,80.1,741.0
2014-01-09 14:00:00+00:00,713.0,488.0,NaN,NaN,992.6,NaN,11.1,77.1,713.0
2014-01-09 15:00:00+00:00,734.0,506.0,NaN,NaN,991.3,NaN,11.6,75.5,734.0
...,...,...,...,...,...,...,...,...,...
2020-06-09 19:00:00+00:00,1034.0,521.0,NaN,NaN,1010.0,NaN,25.6,41.3,1034.0
2020-06-09 20:00:00+00:00,952.0,524.0,NaN,NaN,1010.2,NaN,23.4,40.6,952.0
2020-06-09 21:00:00+00:00,924.0,473.0,NaN,NaN,1010.4,NaN,21.8,40.6,924.0


In [ ]:
calib_df = pd.read_csv(
    Path.cwd() / "example_data" / "FSCD001_calibration.csv"
)
data_hub.calibration_samples_data = calib_df

The `attach_nmdb_data()` method has defaults. Running the below code, without selecting values, will work too. 


```python
data_hub.attach_nmdb_data()
```



In [ ]:
data_hub.attach_nmdb_data(
    station = "JUNG",
    new_column_name = "incoming_neutron_intensity",
    resolution = "60",
    nmdb_table = "revori"
    )
data_hub.crns_data_frame

In [ ]:
data_hub.prepare_static_values()
data_hub.crns_data_frame

You can use your IDE (e.g., in VSCode) to check which methods are available using the 

In [ ]:
from neptoon.quality_control import WhatParamsDoINeed

WhatParamsDoINeed(QAMethod.SPIKE_UNILOF)

The parameters are optional and uses defaults, I'll stick with these.

In [ ]:
qa_flags = QualityAssessmentFlagBuilder()
qa_flags.add_check(
    
    QualityCheck(
        target=QATarget.RELATIVE_HUMIDITY,
        method=QAMethod.RANGE_CHECK,
        parameters={
            "min":0,
            "max":100
                }),

    QualityCheck(
        target=QATarget.RAW_EPI_NEUTRONS,
        method=QAMethod.SPIKE_UNILOF,
    )
)

data_hub.add_quality_flags(custom_flags=qa_flags)
data_hub.apply_quality_flags()
data_hub.flags_data_frame


In [ ]:
data_hub.select_correction(
    correction_type=CorrectionType.INCOMING_INTENSITY,
    correction_theory=CorrectionTheory.ZREDA_2012,
)
data_hub.select_correction(
    correction_type=CorrectionType.HUMIDITY,
    correction_theory=CorrectionTheory.ROSOLEM_2013
)

data_hub.select_correction(
    correction_type=CorrectionType.PRESSURE,
)

# data_hub.select_correction(
#     correction_type=CorrectionType.ABOVE_GROUND_BIOMASS
# )

data_hub.correct_neutrons()
data_hub.crns_data_frame

We've pulled some calibration data that doesn't exactly follow the expected naming convention in neptoon. We can change this in the `CalibrationConfiguration` by updating the expected column names for key columns. 

In [ ]:
calibration_config = CalibrationConfiguration(
    date_time_column_name='DateTime_utc',
    profile_id_column='Profile_ID',
    distance_column='Distance_to_CRNS_m',
    sample_depth_column='Profile_Depth_cm',
    soil_moisture_gravimetric_column='SoilMoisture_g_g',
    bulk_density_of_sample_column='DryBulkDensity_g_cm3',
    soil_organic_carbon_column='SoilOrganicCarbon_g_g',
    lattice_water_column='LatticeWater_g_g',
)

data_hub.calibrate_station(config=calibration_config)
data_hub.calibrator.return_calibration_results_data_frame()

In [ ]:
data_hub.produce_soil_moisture_estimates()
data_hub.crns_data_frame

In [ ]:
data_hub.save_data(append_yaml_hash_to_folder_name=True)